In [1]:
import pandas as pd
from pathlib import Path
import IMP 
import IMP.atom
import IMP.rmf
import RMF
import random
import numpy as np
import matplotlib.pyplot as plt

import IMP
import IMP.core
import IMP.atom
import IMP.container
import IMP.isd
import IMP.pmi.tools
from operator import itemgetter
import math

In [2]:
gsms_dir = Path("/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit")
sampcon_job_dir = Path("/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/3")

# gsms_dir = Path("/Users/matthew/Downloads/116/1000_edit")
# sampcon_job_dir = Path("/Users/matthew/Downloads/116/3")

cluster = 0
cluster_dir = Path(sampcon_job_dir, "cluster.{}".format(cluster))

# Script needs to read in the number of structures contained in the A.rmf3 file because the structures in sampcon cluster A are indexed between 0:N_A-1 while structures in cluster B are indexed between N_A:N_A+N_B-1.
with open(Path(gsms_dir, "A.txt"), 'r') as fp:
    N_A = len(fp.readlines())
    print("N_A: {}".format(N_A))

samp_clust_file = Path(sampcon_job_dir, "cluster.{}.all.txt".format(cluster))
samp_clust_df = pd.read_csv(samp_clust_file, header=None)
# samp_clust_df = samp_clust_df.iloc[:1]

print(samp_clust_file)
print(len(samp_clust_df))

models = list()
for i in range(len(samp_clust_df)):
    frame_num = samp_clust_df.iloc[i,0]
    if frame_num < N_A:
        rmf_file = Path(gsms_dir, "A.rmf3")
    else:
        rmf_file = Path(gsms_dir, "B.rmf3")
        frame_num = frame_num - N_A
        
    models.append((rmf_file, frame_num)) 

    

N_A: 949
/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/3/cluster.0.all.txt
1923


In [3]:
def apply_conn(first, last): 
    apply_restraint = True

    # Apply connectivity runless ALL of the following are true:
    #   - first and last both have RigidBodyMember decorators
    #   - first and last are both RigidMembers
    #   - first and last are part of the same RigidBody object

    # Check for both in a rigid body
    if IMP.core.RigidBodyMember.get_is_setup(first) and \
            IMP.core.RigidBodyMember.get_is_setup(last) and \
            IMP.core.RigidMember.get_is_setup(first) and \
            IMP.core.RigidMember.get_is_setup(last):
        # Check if the rigid body objects for each particle are
        # the same object.
        #  if so, skip connectivity restraint
        if IMP.core.RigidBodyMember(first).get_rigid_body() \
                == IMP.core.RigidBodyMember(last).get_rigid_body():
            apply_restraint = False
    
    return apply_restraint

In [99]:
mols_dict = {0: "MTOR", 2: "RICTOR", 4: "MLST8", 6: "MSIN1", 8: "AKT1"}
conn_dict = dict() 
kappa = 10

for rmf_file, frame_num in models[1:]: 
    print(rmf_file, frame_num)

    fh = RMF.open_rmf_file_read_only(str(rmf_file))
    m = IMP.Model()
    h = IMP.rmf.create_hierarchies(fh, m)[0]
    IMP.rmf.load_frame(fh, RMF.FrameID(frame_num))
    for i in range(0,10,2):
        h_mol = h.get_children()[0].get_children()[i]

        hiers = IMP.pmi.tools.input_adaptor(h_mol, 10)
        # hiers = hiers[0]
        hiers = hiers[0]
        
        SortedSegments = []
        for h_tmp in hiers:
            try:
                start = IMP.atom.Hierarchy(h_tmp).get_children()[0]
            except:  # noqa: E722
                start = IMP.atom.Hierarchy(h_tmp)

            try:
                end = IMP.atom.Hierarchy(h_tmp).get_children()[-1]
            except:  # noqa: E722
                end = IMP.atom.Hierarchy(h_tmp)
            startres = IMP.pmi.tools.get_residue_indexes(start)[0]

            SortedSegments.append((start, end, startres))

        SortedSegments = sorted(SortedSegments, key=itemgetter(2))
        for j in range(len(SortedSegments)-1): 
            last = SortedSegments[j][0]
            first = SortedSegments[j+1][0]
            
            endres = IMP.pmi.tools.get_residue_indexes(last)[-1]
#             print(mols_dict[i], startres, endres)
            
            if apply_conn(first,last):                 
                nreslast = len(IMP.pmi.tools.get_residue_indexes(last))
                lastresn = IMP.pmi.tools.get_residue_indexes(last)[-1]
                nresfirst = len(IMP.pmi.tools.get_residue_indexes(first))
                firstresn = IMP.pmi.tools.get_residue_indexes(first)[0]
                entry = "{},{},{}".format(mols_dict[i], lastresn, firstresn)
                
                residuegap = np.abs(firstresn - lastresn - 1)

                optdist = (0.0 + (float(residuegap) + 1.0) * 3.6) * 1
                hu = IMP.core.HarmonicUpperBound(optdist, kappa)

                dps = IMP.core.SphereDistancePairScore(hu)
                
                pt0 = last.get_particle()
                pt1 = first.get_particle()
#                 sphere_dist = math.sqrt(dps.evaluate_index(m, (pt0.get_index(), pt1.get_index()), None) * 2 / 10)
                
                xyz_0 = IMP.core.XYZR(pt0)
                xyz_1 = IMP.core.XYZR(pt1)
        
#                 r_0 = xyz_0.get_radius()
#                 r_1 = xyz_1.get_radius()
                
#                 d = IMP.algebra.distance(xyz_0,xyz_1)
        
                print(IMP.algebra.get_distance(xyz_0.get_sphere(), xyz_1.get_sphere()) - 3.6)

#                 r = IMP.core.PairRestraint(
#                     m, dps, (pt0.get_index(), pt1.get_index()))

#                 conn_r  = r.evaluate(False)        
#                 print(entry, sphere_dist)
                if entry not in conn_dict.keys(): 
                    conn_dict[entry] = [sphere_dist]
                else: 
                    conn_dict[entry].append(sphere_dist)
                    
#             break 
#         break 
    break 
                    

/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 1
-1.6966578623390416
-0.15526576282781201
-2.082751995863758
-1.3996605948948795
-1.9016681952303078
-1.0083814631143384
-0.9160355988744429
-0.1177314140182717
-3.258449149373462
-4.339075640649375
-1.4959828033341052
-1.7661047320175585
-0.6735461191643624
-1.3979025301539765
-1.3259384749670162
-0.5231426860729695
-2.725582235409146
-1.7341560032529793
-2.8442065653566098
0.24597010495434146
-4.2145558978308575
-0.941090246595389
-2.631713424950997
-3.2452532283774063
-2.9461652307592208
-2.750152629672009
-3.6308786197804648
-4.126617806918102
-0.8025049515541505
-2.4086146394254144
-4.02878861974253
-2.439904904729945
-2.711565901488069
-0.9280435611058615
-3.66197582135896
-3.742131529649701
0.5274159174032227
-0.024500279429214178
0.28274822526331844
0.2875725341301112
0.4286713404521847
-3.1826850458818403
-1.387726723634334
-2.4286366695891712
-3.6907867184955703
-1.3888077024192982


In [96]:
print(IMP.core.XYZR(pt0).get_radius())


5.991901397705078


In [68]:
conn_df = pd.DataFrame(conn_dict)
conn_df.to_csv(Path(Path.home(), "mtorc2/manuscript/submission/models/116_693502_3/cluster.0/conn.csv"))

In [69]:
conn_df.head()

,"MTOR,10,11","MTOR,16,17","MTOR,30,31","MTOR,36,37","MTOR,53,54","MTOR,59,60","MTOR,74,75","MTOR,81,82","MTOR,246,247","MTOR,256,257",...,"AKT1,234,235","AKT1,298,299","AKT1,308,309","AKT1,312,313","AKT1,429,430","AKT1,439,440","AKT1,449,450","AKT1,459,460","AKT1,469,470","AKT1,479,480"
0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.237555,0.0,0.0,0.0,...,0.0,0.0,0.319865,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000
1,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.173722,0.0,0.00000
2,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.08975
3,0.0,0.000000,0.0,0.087292,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.227095,0.0,0.0,0.0,0.000000,0.0,0.00000
4,0.0,0.112976,0.0,0.000000,0.0,0.0,0.332680,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.00000


In [131]:
print(len(conn_df))

1923


In [139]:
# std_dev = math.sqrt(1.380649 * 10**(-23) * 310 / 10) * 10**10
std_dev = .325

sat_df = conn_df > (std_dev * 3)
sat_df = sat_df.astype(int)
sat_df.head()

,"MTOR,10,11","MTOR,16,17","MTOR,30,31","MTOR,36,37","MTOR,53,54","MTOR,59,60","MTOR,74,75","MTOR,81,82","MTOR,246,247","MTOR,256,257",...,"AKT1,234,235","AKT1,298,299","AKT1,308,309","AKT1,312,313","AKT1,429,430","AKT1,439,440","AKT1,449,450","AKT1,459,460","AKT1,469,470","AKT1,479,480"
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [140]:
for i in range(len(sat_df)): 
    print(i, sat_df.iloc[i].sum())

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 0
24 0
25 0
26 0
27 0
28 0
29 0
30 0
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 0
46 1
47 0
48 0
49 0
50 0
51 0
52 1
53 0
54 0
55 0
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 1
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 1
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0
101 0
102 0
103 0
104 0
105 0
106 0
107 0
108 0
109 0
110 0
111 0
112 0
113 0
114 0
115 0
116 0
117 0
118 1
119 1
120 0
121 0
122 0
123 0
124 0
125 0
126 0
127 0
128 0
129 0
130 0
131 0
132 0
133 0
134 0
135 0
136 0
137 0
138 0
139 0
140 0
141 0
142 0
143 0
144 0
145 0
146 1
147 0
148 0
149 0
150 0
151 0
152 0
153 0
154 0
155 0
156 0
157 0
158 0
159 0
160 0
161 0
162 0
163 0
164 0
165 0
166 0
167 0
168 0
169 0
170 0
171 0
172 0
173 0
174 0
175 0
176 0
177 0
178 0
179 0
180 0
181 0
182 0
183 0
184 0


In [141]:
avg_viol = 0 
num_viol = 0
for col in sat_df.columns: 
    if col not in ["MTOR,1222,1223", "MTOR,1232,1233", "MTOR,1242,1243", "MTOR,1252,1253", "MTOR,1260,1261"]: 
        avg_viol = avg_viol + sat_df[col].sum()
        print(col, sat_df[col].sum())
        if sat_df[col].sum() > 0: 
            num_viol = num_viol + 1
  
print(num_viol)
# print(avg_viol / len(sat_df.columns))

MTOR,10,11 0
MTOR,16,17 0
MTOR,30,31 0
MTOR,36,37 0
MTOR,53,54 0
MTOR,59,60 0
MTOR,74,75 0
MTOR,81,82 0
MTOR,246,247 0
MTOR,256,257 0
MTOR,257,258 0
MTOR,289,290 0
MTOR,299,300 0
MTOR,309,310 0
MTOR,319,320 0
MTOR,329,330 0
MTOR,339,340 0
MTOR,349,350 0
MTOR,355,356 0
MTOR,380,381 0
MTOR,385,386 0
MTOR,404,405 1
MTOR,409,410 0
MTOR,466,467 0
MTOR,476,477 0
MTOR,477,478 0
MTOR,549,550 0
MTOR,559,560 0
MTOR,569,570 0
MTOR,577,578 0
MTOR,633,634 0
MTOR,643,644 0
MTOR,903,904 0
MTOR,913,914 0
MTOR,923,924 0
MTOR,932,933 0
MTOR,1814,1815 0
MTOR,1824,1825 0
MTOR,1834,1835 0
MTOR,1844,1845 0
MTOR,1854,1855 0
MTOR,1864,1865 0
MTOR,1866,1867 0
MTOR,2436,2437 0
MTOR,2446,2447 0
MTOR,2456,2457 0
MTOR,2466,2467 0
MTOR,2476,2477 0
MTOR,2486,2487 0
MTOR,2491,2492 0
RICTOR,10,11 0
RICTOR,20,21 0
RICTOR,26,27 0
RICTOR,504,505 0
RICTOR,514,515 0
RICTOR,524,525 0
RICTOR,534,535 0
RICTOR,857,858 0
RICTOR,867,868 0
RICTOR,871,872 0
RICTOR,999,1000 0
RICTOR,1009,1010 0
RICTOR,1019,1020 1
RICTOR,1029,1030 0

In [84]:
col = "AKT1,228,229"
for i in range(len(conn_df[col])): 
    print(i, conn_df[col].iloc[i])

0 0.0
1 0.0
2 0.45326487608887106
3 0.0
4 0.07095245550871487
5 0.0
6 0.0
7 0.26971605659582165
8 0.22639345614362005
9 0.0
10 0.09861368049446417
11 0.0
12 0.5194536983788924
13 0.0
14 0.06525998472495464
15 0.0
16 0.0
17 0.0
18 0.13842593340228104
19 0.38163002714892924
20 0.0
21 0.047861314998420834
22 0.0
23 0.0
24 0.42812197495214255
25 0.0
26 0.4513283173047653
27 0.0
28 0.42935683294874805
29 0.0
30 0.0
31 0.6480200693703941
32 0.31717703056583835
33 0.0
34 0.0
35 0.0
36 0.0
37 0.0
38 0.0
39 0.0
40 0.3751640087302426
41 0.0
42 0.0
43 0.25901929996059314
44 0.0
45 0.12542693367276625
46 0.19871588822835973
47 0.0
48 0.0
49 0.0
50 0.2999297598638342
51 0.0
52 0.0
53 0.0
54 0.0
55 0.0
56 0.26792003938793085
57 0.1341364775305709
58 0.0
59 0.0
60 0.0
61 0.0
62 0.0
63 0.0
64 0.40495535953867856
65 0.0
66 0.0
67 0.0
68 0.0
69 0.4944336454455409
70 0.0
71 0.0
72 0.0
73 0.0
74 0.6703841581613044
75 0.0
76 0.035125387356639504
77 0.21538600938521624
78 0.0
79 0.01956326811436382
80 0.442

In [86]:
std_dev * 3

0.6105532163538245

In [83]:
print(conn_df[col] > (std_dev * 3))

0       False
1       False
2       False
3       False
4       False
        ...  
1918    False
1919    False
1920    False
1921    False
1922    False
Name: AKT1,479,480, Length: 1923, dtype: bool


In [66]:
SortedSegments

[("Fragment [1-11)" Fragment: [1, 11)   rigid member: RB9475(122.282 50.525 208.533: 6.12156),
  "Fragment [1-11)" Fragment: [1, 11)   rigid member: RB9475(122.282 50.525 208.533: 6.12156),
  1),
 ("Fragment [11-21)" Fragment: [11, 21)   rigid member: RB9475(135.638 69.0412 203.816: 6.63533),
  "Fragment [11-21)" Fragment: [11, 21)   rigid member: RB9475(135.638 69.0412 203.816: 6.63533),
  11),
 ("Fragment [21-31)" Fragment: [21, 31)   rigid member: RB9475(129.496 65.0198 206.803: 6.76119),
  "Fragment [21-31)" Fragment: [21, 31)   rigid member: RB9475(129.496 65.0198 206.803: 6.76119),
  21),
 ("Fragment [31-41)" Fragment: [31, 41)   rigid member: RB9475(125.348 58.7725 204.348: 6.27872),
  "Fragment [31-41)" Fragment: [31, 41)   rigid member: RB9475(125.348 58.7725 204.348: 6.27872),
  31),
 ("Fragment [41-51)" Fragment: [41, 51)   rigid member: RB9475(114.833 62.6953 204.951: 6.36665),
  "Fragment [41-51)" Fragment: [41, 51)   rigid member: RB9475(114.833 62.6953 204.951: 6.36665),

In [42]:
last

"1-10_bead" Fragment: [1, 11)  (186.014 170.24 116.055: 5.9919)

In [61]:
conn_dict

{'MTOR,10,11': [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.009999641939449244,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.6074999064014448,
  0.1924631687086218,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.11026749289060467,
  0.0,
  0.0,
  0.0,
  0.3810467772208521,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.03159245510212072,
  0.13791165611532596,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.06977774604395348,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 'MTOR,16,17': [0.0,
  0.0,
  0.0,
  0.0,
  0.11297630265434712,
  0.0,
  0.0,
  0.0,
  0.0,
  0.12875597446770426,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.2133

In [62]:
pd.DataFrame(conn_dict)

,"MTOR,10,11","MTOR,16,17","MTOR,30,31","MTOR,36,37","MTOR,53,54","MTOR,59,60","MTOR,74,75","MTOR,81,82","MTOR,246,247","MTOR,256,257",...,"AKT1,234,235","AKT1,298,299","AKT1,308,309","AKT1,312,313","AKT1,429,430","AKT1,439,440","AKT1,449,450","AKT1,459,460","AKT1,469,470","AKT1,479,480"
0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.237555,0.000000,0.0,0.0,...,0.000000,0.0,0.319865,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000
1,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.173722,0.000000,0.00000
2,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.08975
3,0.0,0.000000,0.0,0.087292,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.227095,0.0,0.0,0.000000,0.000000,0.000000,0.00000
4,0.0,0.112976,0.0,0.000000,0.000000,0.0,0.332680,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.189049,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000
96,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.274288,0.0,0.0,...,0.134643,0.0,0.000000,0.149134,0.0,0.0,0.648027,0.000000,0.134968,0.00000
97,0.0,0.000000,0.0,0.000000,0.345400,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000
98,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.336570,0.000000,0.00000


In [27]:
SortedSegments[1]

("11-16_bead" Fragment: [11, 17)  (194.78 178.595 116.453: 5.05377),
 "11-16_bead" Fragment: [11, 17)  (194.78 178.595 116.453: 5.05377),
 11)

In [14]:
r.evaluate(False)

15163.91475904349

In [ ]:
h_mol = h.get_children()[0].get_children()[0]

hiers = IMP.pmi.tools.input_adaptor(h_mol, 10)
# hiers = hiers[0]
hiers = hiers[0]

kappa = 10  # spring constant used for the harmonic restraints
SortedSegments = []
for h in hiers:
    try:
        start = IMP.atom.Hierarchy(h).get_children()[0]
    except:  # noqa: E722
        start = IMP.atom.Hierarchy(h)

    try:
        end = IMP.atom.Hierarchy(h).get_children()[-1]
    except:  # noqa: E722
        end = IMP.atom.Hierarchy(h)

    startres = IMP.pmi.tools.get_residue_indexes(start)[0]
    endres = IMP.pmi.tools.get_residue_indexes(end)[-1]
    print(startres, endres)
    SortedSegments.append((start, end, startres))
    
SortedSegments = sorted(SortedSegments, key=itemgetter(2))

In [ ]:
print(SortedSegments)

In [ ]:

for x in range(len(SortedSegments) - 1):

    last = SortedSegments[x][1]
    first = SortedSegments[x + 1][0]
    
    apply_restraint = True

    # Apply connectivity runless ALL of the following are true:
    #   - first and last both have RigidBodyMember decorators
    #   - first and last are both RigidMembers
    #   - first and last are part of the same RigidBody object

    # Check for both in a rigid body
    if IMP.core.RigidBodyMember.get_is_setup(first) and \
            IMP.core.RigidBodyMember.get_is_setup(last) and \
            IMP.core.RigidMember.get_is_setup(first) and \
            IMP.core.RigidMember.get_is_setup(last):
        # Check if the rigid body objects for each particle are
        # the same object.
        #  if so, skip connectivity restraint
        if IMP.core.RigidBodyMember(first).get_rigid_body() \
                == IMP.core.RigidBodyMember(last).get_rigid_body():
            apply_restraint = False
    
    if apply_restraint: 
        scores_dict[(str(last), str(first))] = list() 

In [ ]:
print(scores_dict)

In [28]:
kappa = 10  # spring constant used for the harmonic restraints

# connect the particles

for rmf_file, frame_num in models: 
    print(rmf_file, frame_num)

    fh = RMF.open_rmf_file_read_only(str(rmf_file))
    m = IMP.Model()
    h = IMP.rmf.create_hierarchies(fh, m)[0]
    IMP.rmf.load_frame(fh, RMF.FrameID(frame_num))
    
    h_mol = h.get_children()[0].get_children()[0]

    hiers = IMP.pmi.tools.input_adaptor(h_mol, 10)
    # hiers = hiers[0]
    hiers = hiers[0]

    kappa = 10  # spring constant used for the harmonic restraints
    SortedSegments = []
    for h in hiers:
        try:
            start = IMP.atom.Hierarchy(h).get_children()[0]
        except:  # noqa: E722
            start = IMP.atom.Hierarchy(h)

        try:
            end = IMP.atom.Hierarchy(h).get_children()[-1]
        except:  # noqa: E722
            end = IMP.atom.Hierarchy(h)

        startres = IMP.pmi.tools.get_residue_indexes(start)[0]
        SortedSegments.append((start, end, startres))
    SortedSegments = sorted(SortedSegments, key=itemgetter(2))
    
    for (last, first) in scores_dict.keys():
        nreslast = len(IMP.pmi.tools.get_residue_indexes(last))
        lastresn = IMP.pmi.tools.get_residue_indexes(last)[-1]
        nresfirst = len(IMP.pmi.tools.get_residue_indexes(first))
        firstresn = IMP.pmi.tools.get_residue_indexes(first)[0]

        residuegap = firstresn - lastresn - 1

        optdist = (0.0 + (float(residuegap) + 1.0) * 3.6) * 1
        hu = IMP.core.HarmonicUpperBound(optdist, kappa)

        dps = IMP.core.SphereDistancePairScore(hu)

        pt0 = last.get_particle()
        pt1 = first.get_particle()
        r = IMP.core.PairRestraint(
            m, dps, (pt0.get_index(), pt1.get_index()))

        conn_r  = r.evaluate(False)
        conn_s = math.e ** (-conn_r)

        scores_dict[(last, first)].append(conn_r)
    #         print(conn_r, conn_s)


/wynton/group/sali/mhancock/mtorc2/exp_13/116/analysis/693502/sampcon_4/gsms/1000_edit/A.rmf3 0


NameError: name 'scores_dict' is not defined

In [33]:
kappa = 10  # spring constant used for the harmonic restraints

# connect the particles
particle_pairs = []
for x in range(len(SortedSegments) - 1):

    last = SortedSegments[x][1]
    first = SortedSegments[x + 1][0]
    
    apply_restraint = True

    # Apply connectivity runless ALL of the following are true:
    #   - first and last both have RigidBodyMember decorators
    #   - first and last are both RigidMembers
    #   - first and last are part of the same RigidBody object

    # Check for both in a rigid body
    if IMP.core.RigidBodyMember.get_is_setup(first) and \
            IMP.core.RigidBodyMember.get_is_setup(last) and \
            IMP.core.RigidMember.get_is_setup(first) and \
            IMP.core.RigidMember.get_is_setup(last):
        # Check if the rigid body objects for each particle are
        # the same object.
        #  if so, skip connectivity restraint
        if IMP.core.RigidBodyMember(first).get_rigid_body() \
                == IMP.core.RigidBodyMember(last).get_rigid_body():
            apply_restraint = False

    if apply_restraint:

        nreslast = len(IMP.pmi.tools.get_residue_indexes(last))
        lastresn = IMP.pmi.tools.get_residue_indexes(last)[-1]
        nresfirst = len(IMP.pmi.tools.get_residue_indexes(first))
        firstresn = IMP.pmi.tools.get_residue_indexes(first)[0]

        residuegap = firstresn - lastresn - 1
        print(residuegap)
        optdist = (0.0 + (float(residuegap) + 1.0) * 3.6) * 1
        hu = IMP.core.HarmonicUpperBound(optdist, kappa)

        dps = IMP.core.SphereDistancePairScore(hu)

        pt0 = last.get_particle()
        pt1 = first.get_particle()
        particle_pairs.append((pt0, pt1))
        r = IMP.core.PairRestraint(
            m, dps, (pt0.get_index(), pt1.get_index()))
        
        print(last, first)
        conn_r  = r.evaluate(False)
        conn_s = math.e ** (-conn_r)
        
#         scores.append(conn_r)
        print(conn_r, conn_s)
    
    break


0
"1-10_bead" Fragment: [1, 11)  (186.014 170.24 116.055: 5.9919) "11-16_bead" Fragment: [11, 17)  (194.78 178.595 116.453: 5.05377)
0.0 1.0


In [ ]:
r.evaluate(False)

In [ ]:
dps.evaluate()

In [ ]:
math.e ** (-0.7433790613956471)

In [34]:
residuegap

0

In [35]:
firstresn

11

In [36]:
lastresn

10

In [37]:
last

"1-10_bead" Fragment: [1, 11)  (186.014 170.24 116.055: 5.9919)

In [38]:
first

"11-16_bead" Fragment: [11, 17)  (194.78 178.595 116.453: 5.05377)